## Cargar Dataframes


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col, lit
from pyspark.sql.functions import col, count, avg, sum, max, min, when, datediff, current_date
import re

In [2]:
# ==============================
# 1. Inicializar Spark Session
# ==============================
spark = SparkSession.builder \
    .appName("HDFS_NiFi_Data_Cleaning") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://namenode:9000") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/30 19:46:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# ==============================
# 2. Paths en HDFS
# ==============================
hdfs_path_maestra = "hdfs://namenode:9000/user/nifi/maestra.csv"
hdfs_path_zona = "hdfs://namenode:9000/user/nifi/bd_zona.csv"

file_familia = "/user/nifi/Resumen_Valores-VENTA_POR_FAMILIA2.csv"
file_producto = "/user/nifi/Resumen_Valores-VENTA_POR_PRODUCTO2.csv"

In [4]:
# Primer CSV
df_maestra = spark.read.csv(hdfs_path_maestra, header=True, inferSchema=True, sep=";")
df_maestra.show(truncate=False)

+-----------+------------------+-------------------------------------------------+
|Producto   |Numero de articulo|Descripcion                                      |
+-----------+------------------+-------------------------------------------------+
|LAGRICEL PF|41582             |LAGRICEL OFTENO LIBRE DE CONSERVADORES (PF) 10 ML|
|ELIPTIC PF |41561             |ELIPTIC OFTENO 5ML PF PERU                       |
|LAGRICEL   |40515             |LAGRICEL OFTENO 0.5 ML                           |
|FLUMETOL NF|40513             |FLUMETOL NF OFTENO 5ML                           |
|TRAZIDEX O |40341             |TRAZIDEX OFTENO 5 ML.                            |
|TRAZIDEX U |40342             |TRAZIDEX UNGENA 3.5 G                            |
|SOPHIPREN  |40338             |SOPHIPREN OFTENO 5 ML                            |
|GAAP       |40498             |GAAP OFTENO 3 ML                                 |
|AQUADRAN   |41945             |AQUADRAN 10G                                     |
|GAA

In [5]:
# Segundo CSV
df_zona = spark.read.csv(hdfs_path_zona, header=True, inferSchema=True, sep=";")
df_zona.show(truncate=False)

+-----------+-----------------------------------------------------------------+-----------+-------+---+-------+--------+
|Vendedor   |Nombre Cliente                                                   |Producto   |MES NUM|Mes|2025   |CANTIDAD|
+-----------+-----------------------------------------------------------------+-----------+-------+---+-------+--------+
|Pharma - N1|ADMINISTRADORA CLINICA TRESA S.A                                 |AGGLAD     |1      |ENE|0      |0       |
|Pharma - N1|ADMINISTRADORA CLINICA TRESA S.A                                 |FLUMETOL NF|1      |ENE|0      |0       |
|Pharma - N1|ADMINISTRADORA CLINICA TRESA S.A                                 |GAAP       |1      |ENE|0      |0       |
|Pharma - N1|ADMINISTRADORA CLINICA TRESA S.A                                 |LAGRICEL   |1      |ENE|0      |0       |
|Pharma - N1|BENEL PEREZ,DENNY JAVIER                                         |FLUMETOL NF|1      |ENE|898.37 |20      |
|Pharma - N1|BENEL PEREZ,DENNY J

### Corrección y adaptación de resumen de valores y unidades


In [6]:
def rename_month_columns(df):
    """
    Renombra los bloques de columnas por mes con las métricas:
    Venta, TGT, PY24, pct
    """
    meses = ["Enero","Febrero","Marzo","Abril","Mayo","Junio",
             "Julio","Agosto","Septiembre","Octubre","Noviembre","Diciembre","YTD_JUL"]
    metricas = ["Venta","TGT","PY24","pct"]

    new_cols = ["Producto_Familia"]  # primera columna fija
    col_idx = 1  # empezamos después de Producto_Familia

    for mes in meses:
        for metrica in metricas:
            if col_idx < len(df.columns):
                new_cols.append(f"{mes}_{metrica}")
                col_idx += 1

    # Si sobran columnas (por ruido en el CSV), las nombramos Extra
    while len(new_cols) < len(df.columns):
        new_cols.append(f"Extra_{len(new_cols)}")

    return df.toDF(*new_cols)

In [7]:
def load_and_clean(path):
    # ============================
    # 1. Leer CSV sin header
    # ============================
    df = spark.read.option("sep", ";").option("header", "false").csv(path)

    # ============================
    # 2. Tomar primera fila como header real
    # ============================
    header_row = df.first()

    # ============================
    # 3. Filtrar esa fila del dataframe
    # ============================
    df = df.filter(col("_c0") != header_row[0])

    # ============================
    # 4. Columnas temporales
    # ============================
    tmp_headers = [f"Col_{i}" for i in range(len(df.columns))]
    df = df.toDF(*tmp_headers)

    # ============================
    # 5. Renombrar columnas a Mes_Métrica
    # ============================
    df = rename_month_columns(df)

    return df

In [8]:
# ==============================
# 6. Aplicar función
# ==============================
df_familia = load_and_clean(file_familia)
df_producto = load_and_clean(file_producto)

25/09/30 19:46:53 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [9]:
df_familia.printSchema()
df_producto.printSchema()

root
 |-- Producto_Familia: string (nullable = true)
 |-- Enero_Venta: string (nullable = true)
 |-- Enero_TGT: string (nullable = true)
 |-- Enero_PY24: string (nullable = true)
 |-- Enero_pct: string (nullable = true)
 |-- Febrero_Venta: string (nullable = true)
 |-- Febrero_TGT: string (nullable = true)
 |-- Febrero_PY24: string (nullable = true)
 |-- Febrero_pct: string (nullable = true)
 |-- Marzo_Venta: string (nullable = true)
 |-- Marzo_TGT: string (nullable = true)
 |-- Marzo_PY24: string (nullable = true)
 |-- Marzo_pct: string (nullable = true)
 |-- Abril_Venta: string (nullable = true)
 |-- Abril_TGT: string (nullable = true)
 |-- Abril_PY24: string (nullable = true)
 |-- Abril_pct: string (nullable = true)
 |-- Mayo_Venta: string (nullable = true)
 |-- Mayo_TGT: string (nullable = true)
 |-- Mayo_PY24: string (nullable = true)
 |-- Mayo_pct: string (nullable = true)
 |-- Junio_Venta: string (nullable = true)
 |-- Junio_TGT: string (nullable = true)
 |-- Junio_PY24: string 

In [10]:
# De forma alargada
from pyspark.sql.functions import expr, regexp_extract, first
from pyspark.sql import functions as F

def unpivot_to_wide(df):
    id_col = "Producto_Familia"
    value_cols = [c for c in df.columns if c != id_col]

    # Crear expresión stack dinámicamente
    expr_str = "stack({0}, {1}) as (Columna, Valor)".format(
        len(value_cols),
        ",".join([f"'{c}', {c}" for c in value_cols])
    )

    # Expandir a formato largo
    df_long = df.select(id_col, expr(expr_str))

    # Separar Mes y Métrica
    df_long = df_long.withColumn("Mes", regexp_extract("Columna", r"^(.*)_(Venta|TGT|PY24|pct)$", 1)) \
                     .withColumn("Metrica", regexp_extract("Columna", r"^(.*)_(Venta|TGT|PY24|pct)$", 2)) \
                     .drop("Columna")

    # Pivotear: convertir Métrica en columnas
    df_wide = df_long.groupBy(id_col, "Mes").pivot("Metrica").agg(F.first("Valor"))

    return df_wide


In [11]:
from pyspark.sql.functions import col

# Quitar la fila "Producto"
df_familia_clean = df_familia.filter(df_familia["Producto_Familia"] != "Producto")
df_producto_clean = df_producto.filter(df_producto["Producto_Familia"] != "Producto")

# Aplicar unpivot corregido (con pivot final)
df_familia_wide = unpivot_to_wide(df_familia_clean)
df_producto_wide = unpivot_to_wide(df_producto_clean)

# Convertir columnas numéricas a double
num_cols = ["Venta", "TGT", "PY24", "pct"]

for c in num_cols:
    df_familia_wide = df_familia_wide.withColumn(c, col(c).cast("double"))
    df_producto_wide = df_producto_wide.withColumn(c, col(c).cast("double"))

# Mostrar resultados
print("=== Familia ===")
df_familia_wide.show(20, truncate=False)

print("=== Producto ===")
df_producto_wide.show(20, truncate=False)


=== Familia ===
+----------------+----------+---------+-----------+---------+-----------+
|Producto_Familia|Mes       |PY24     |TGT        |Venta    |pct        |
+----------------+----------+---------+-----------+---------+-----------+
|AGGLAD          |Abril     |24635.46 |120891.2407|91519.9  |0.757043269|
|AGGLAD          |Agosto    |25769.42 |2560.845455|0.0      |0.0        |
|AGGLAD          |Diciembre |71802.0  |2681.545455|0.0      |0.0        |
|AGGLAD          |Enero     |22534.35 |110094.81  |110094.81|1.0        |
|AGGLAD          |Febrero   |31785.06 |114546.1364|110784.62|0.96716156 |
|AGGLAD          |Julio     |29784.66 |133581.4492|18240.65 |0.136550772|
|AGGLAD          |Junio     |20271.95 |120891.2407|92445.3  |0.764698083|
|AGGLAD          |Marzo     |31785.06 |114546.1364|81102.53 |0.708033745|
|AGGLAD          |Mayo      |26193.67 |120891.2407|80999.82 |0.670022241|
|AGGLAD          |Noviembre |52414.29 |2681.545455|0.0      |0.0        |
|AGGLAD          |Octu

In [12]:
from pyspark.sql.functions import trim, upper

# Normalizar columna Mes en ambos dataframes
df_familia_wide = df_familia_wide.withColumn("Mes", trim(upper(df_familia_wide["Mes"])))
df_producto_wide = df_producto_wide.withColumn("Mes", trim(upper(df_producto_wide["Mes"])))

In [13]:
from pyspark.sql.functions import when, col

# Lista de métricas a convertir
metricas = ["Venta", "TGT", "PY24", "pct"]

# Familia
for m in metricas:
    df_familia_wide = df_familia_wide.withColumn(
        m,
        when(col(m).isNull(), 0.0).otherwise(col(m).cast("double"))
    )

# Producto
for m in metricas:
    df_producto_wide = df_producto_wide.withColumn(
        m,
        when(col(m).isNull(), 0.0).otherwise(col(m).cast("double"))
    )

In [14]:
from pyspark.sql.functions import lit

# Agregar columna de nivel
df_familia_wide = df_familia_wide.withColumn("Nivel", lit("Familia"))
df_producto_wide = df_producto_wide.withColumn("Nivel", lit("Producto"))

# Unir ambos datasets
df_total = (
    df_familia_wide
    .unionByName(df_producto_wide, allowMissingColumns=True)
)

In [15]:
df_total.show(100, truncate=False)

+----------------+----------+----------+-----------+---------+-----------+-------+
|Producto_Familia|Mes       |PY24      |TGT        |Venta    |pct        |Nivel  |
+----------------+----------+----------+-----------+---------+-----------+-------+
|AGGLAD          |ABRIL     |24635.46  |120891.2407|91519.9  |0.757043269|Familia|
|AGGLAD          |AGOSTO    |25769.42  |2560.845455|0.0      |0.0        |Familia|
|AGGLAD          |DICIEMBRE |71802.0   |2681.545455|0.0      |0.0        |Familia|
|AGGLAD          |ENERO     |22534.35  |110094.81  |110094.81|1.0        |Familia|
|AGGLAD          |FEBRERO   |31785.06  |114546.1364|110784.62|0.96716156 |Familia|
|AGGLAD          |JULIO     |29784.66  |133581.4492|18240.65 |0.136550772|Familia|
|AGGLAD          |JUNIO     |20271.95  |120891.2407|92445.3  |0.764698083|Familia|
|AGGLAD          |MARZO     |31785.06  |114546.1364|81102.53 |0.708033745|Familia|
|AGGLAD          |MAYO      |26193.67  |120891.2407|80999.82 |0.670022241|Familia|
|AGG

## Manejo de datos nulos


In [16]:
from pyspark.sql.functions import col

# Contar nulos en df_maestra
df_maestra.select([
    (col(c).isNull().cast("int")).alias(c) for c in df_maestra.columns
]).groupBy().sum().show()

# Contar nulos en df_zona
df_zona.select([
    (col(c).isNull().cast("int")).alias(c) for c in df_zona.columns
]).groupBy().sum().show()


+-------------+-----------------------+----------------+
|sum(Producto)|sum(Numero de articulo)|sum(Descripcion)|
+-------------+-----------------------+----------------+
|            0|                      0|               0|
+-------------+-----------------------+----------------+

+-------------+-------------------+-------------+------------+--------+---------+-------------+
|sum(Vendedor)|sum(Nombre Cliente)|sum(Producto)|sum(MES NUM)|sum(Mes)|sum(2025)|sum(CANTIDAD)|
+-------------+-------------------+-------------+------------+--------+---------+-------------+
|            0|                  0|            0|           0|       0|        0|            0|
+-------------+-------------------+-------------+------------+--------+---------+-------------+



In [17]:
# Eliminar filas con valores nulos en maestra
df_maestra_clean = df_maestra.na.drop()

# Eliminar filas con valores nulos en zona
df_zona_clean = df_zona.na.drop()


In [18]:
# Rellenar con texto en columnas de tipo string
df_maestra_filled = df_maestra.na.fill("Desconocido")

# Rellenar con 0 en columnas numéricas de df_zona
df_zona_filled = df_zona.na.fill(0)


## Correción de tipo de datos


In [19]:
df_maestra = df_maestra.withColumn("Numero de articulo", col("Numero de articulo").cast("int"))

In [20]:
df_zona = df_zona.withColumn("2025", col("2025").cast("double")) 

In [21]:
# Filtrado de datos irrelevantes

In [22]:
# Filtrar productos sin descripción
df_maestra = df_maestra.filter(col("Descripcion").isNotNull())

# Filtrar zonas con ventas registradas en cero
df_zona = df_zona.filter(col("2025") > 0)


## Union


In [23]:
df_zona = df_zona.filter(col("CANTIDAD") >= 0)
df_zona = df_zona.filter(col("2025") >= 0)

In [24]:
from pyspark.sql.functions import col

df_final = df_maestra.join(
    df_zona,
    df_maestra["Producto"] == df_zona["Producto"],
    how="inner"
).drop(df_zona["Producto"])

In [25]:
df_final.show()

+-----------+------------------+--------------------+-----------+--------------------+-------+---+-------+--------+
|   Producto|Numero de articulo|         Descripcion|   Vendedor|      Nombre Cliente|MES NUM|Mes|   2025|CANTIDAD|
+-----------+------------------+--------------------+-----------+--------------------+-------+---+-------+--------+
|FLUMETOL NF|             40513|FLUMETOL NF OFTEN...|Pharma - N1|BENEL PEREZ,DENNY...|      1|ENE| 898.37|      20|
| TRAZIDEX U|             40342|TRAZIDEX UNGENA 3...|Pharma - N1|BENEL PEREZ,DENNY...|      1|ENE|1028.61|      20|
|FLUMETOL NF|             40513|FLUMETOL NF OFTEN...|Pharma - N1|BM CLINICA DE OJO...|      1|ENE| 612.53|      15|
|       GAAP|             40498|    GAAP OFTENO 3 ML|Pharma - N1|BM CLINICA DE OJO...|      1|ENE|1925.85|      25|
| TRAZIDEX O|             40341|TRAZIDEX OFTENO 5...|Pharma - N1|BM CLINICA DE OJO...|      1|ENE| 250.05|       6|
| TRAZIDEX U|             40342|TRAZIDEX UNGENA 3...|Pharma - N1|BM CLIN

## Feature engeeniring

In [30]:
from pyspark.sql.functions import col, count, avg, sum, max, min, when, datediff, current_date


In [31]:
df_final = df_final.withColumn("Precio_Unitario", 
                               when(col("CANTIDAD") > 0, col("2025") / col("CANTIDAD")).otherwise(0))

In [32]:
df_final = df_final.withColumn("Precio_Unitario", 
                               when(col("CANTIDAD") > 0, col("2025") / col("CANTIDAD")).otherwise(0))

In [33]:
# 2. Gasto total por cliente
gasto_cliente = df_final.groupBy("Nombre Cliente").agg(sum("2025").alias("Gasto_Total"))

In [34]:
# 3. Cantidad total por cliente
cantidad_cliente = df_final.groupBy("Nombre Cliente").agg(sum("CANTIDAD").alias("Cantidad_Total"))

In [35]:
# 4. Promedio de gasto y cantidad por cliente
promedios_cliente = df_final.groupBy("Nombre Cliente").agg(
    avg("2025").alias("Gasto_Promedio"),
    avg("CANTIDAD").alias("Cantidad_Promedio")
)

# 5. Frecuencia de compra por cliente (número de transacciones)
frecuencia_cliente = df_final.groupBy("Nombre Cliente").agg(
    count("Producto").alias("Frecuencia_Compra")
)

# 6. Última compra (para calcular recencia)
ultima_compra = df_final.groupBy("Nombre Cliente").agg(
    max("MES NUM").alias("Ultimo_Mes")
)

In [36]:
# Combinar todas las features
features = gasto_cliente \
    .join(cantidad_cliente, "Nombre Cliente") \
    .join(promedios_cliente, "Nombre Cliente") \
    .join(frecuencia_cliente, "Nombre Cliente") \
    .join(ultima_compra, "Nombre Cliente")

# Mostrar resultado
features.show(10, truncate=False)

+-----------------------------------------+------------------+--------------+------------------+------------------+-----------------+----------+
|Nombre Cliente                           |Gasto_Total       |Cantidad_Total|Gasto_Promedio    |Cantidad_Promedio |Frecuencia_Compra|Ultimo_Mes|
+-----------------------------------------+------------------+--------------+------------------+------------------+-----------------+----------+
|CLINICA SAN FELIPE S A                   |21985.35          |700.0         |1099.2675         |35.0              |20               |7         |
|GOOD VISION E.I.R.L.                     |572.59            |12.0          |572.59            |12.0              |1                |3         |
|FONDO DE EMPLEADOS DEL BANCO DE LA NACION|8872.98           |229.0         |2957.66           |76.33333333333333 |3                |3         |
|CONSORCIO QORITAQQUE B Y T S.R.L.        |1427.8200000000002|29.0          |475.94000000000005|9.666666666666666 |3              

In [47]:
# Agregar df_total por producto (suma o promedio de todos los meses)
df_total_agg = df_total.groupBy("Producto_Familia", "Nivel").agg(
    sum("Venta").alias("Venta_Total_Anual"),
    sum("TGT").alias("Target_Total_Anual"),
    sum("PY24").alias("PY24_Total_Anual"),
    avg("pct").alias("pct_Promedio")
)

df_completo = df_final.join(
    df_total_agg,
    df_final["Producto"] == df_total_agg["Producto_Familia"],
    how="left"
)

In [49]:
df_completo.show()

+-----------+------------------+--------------------+-----------+--------------------+-------+---+-------+--------+------------------+----------------+--------+------------------+------------------+------------------+--------------------+
|   Producto|Numero de articulo|         Descripcion|   Vendedor|      Nombre Cliente|MES NUM|Mes|   2025|CANTIDAD|   Precio_Unitario|Producto_Familia|   Nivel| Venta_Total_Anual|Target_Total_Anual|  PY24_Total_Anual|        pct_Promedio|
+-----------+------------------+--------------------+-----------+--------------------+-------+---+-------+--------+------------------+----------------+--------+------------------+------------------+------------------+--------------------+
|FLUMETOL NF|             40513|FLUMETOL NF OFTEN...|Pharma - N1|BENEL PEREZ,DENNY...|      1|ENE| 898.37|      20|           44.9185|     FLUMETOL NF|Producto|         1621334.0|     1593594.73592|1884900.4700000002| 0.10436897615384616|
|FLUMETOL NF|             40513|FLUMETOL NF 

In [50]:
# Contar registros antes y después
print(f"Registros en df_final: {df_final.count()}")
print(f"Registros en df_completo: {df_completo.count()}")

# Deberían ser iguales si el join está bien hecho

Registros en df_final: 3840
Registros en df_completo: 5556
